In [1]:
%load_ext autoreload
%autoreload 2
%pwd

'/home/luca/uni/master/dl-in-vc/voxl3r'

In [2]:
# setup PYTHONPATH
import sys
sys.path += ['.', './extern/scannetpp', './extern/mast3r', './extern/mast3r/dust3r']

In [3]:
from pathlib import Path
import torch
from torchvision.io import read_image

import pyvista as pv
pv.set_jupyter_backend('client')

In [4]:
pv.start_xvfb()

In [5]:
import os


ckpt_folder = list(
        Path("./.lightning/surface-net-3d/surface-net-3d/").glob("*")
    )
ckpt_folder = sorted(ckpt_folder, key=os.path.getmtime)
last_ckpt_folder = ckpt_folder[-1]
run_name = last_ckpt_folder.stem
print(f"Last training is {run_name}")


Last training is bj7r6o9k


In [6]:
import glob


base_dir = '/home/luca/mnt/data/scannetpp/data'
pattern = os.path.join(base_dir, '*', 'prepared_grids', 'dslr', '*furthest_center_1.47')
matching_paths = glob.glob(pattern)

# Extract the parent directories (two levels up from 'undistorted_images')
scenes = [os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(path)))) for path in matching_paths]
    

In [12]:
for path in matching_paths:
    if Path(path).exists() is False:
        print(path)
        continue


In [7]:
import os
from experiments.surface_net_3d.data import SurfaceNet3DDataConfig
from experiments.surface_net_3d.run import visualize_run
from utils.data_parsing import load_yaml_munch

config = load_yaml_munch("./utils/config.yaml")
data_config = SurfaceNet3DDataConfig(data_dir=config.data_dir, batch_size=16, num_workers=1, with_furthest_displacement=True, scenes=scenes, concatinate_pe=True)


visualize_run(run_name, 500, 101, show = ["val"], data_config=data_config)

ImportError: cannot import name 'create_chunk' from partially initialized module 'utils.chunking' (most likely due to a circular import) (/home/luca/uni/master/dl-in-vc/voxl3r/utils/chunking.py)

In [ ]:
import numpy as np
from dataset import SceneDataset
from experiments.surface_net_3d.data import SurfaceNet3DDataConfig
from utils.chunking import create_chunk, mesh_2_voxels
from utils.data_parsing import load_yaml_munch
from utils.transformations import invert_pose
from utils.visualize import visualize_mesh

scene = "4c5c60fa76"
image_name = "DSC09946.JPG"


resolution = data_config.grid_resolution
grid_size = data_config.grid_size
max_seq_len = data_config.seq_len

chunk_size = resolution * grid_size.astype(np.float32)
center = np.array([0.0, 0.0, chunk_size[2]])

dataset = SceneDataset(
            camera=data_config.camera,
            data_dir=data_config.data_dir,
            scenes=data_config.scenes,
        )

idx = dataset.get_index_from_scene(scene)

data = dataset[idx]
mesh = data["mesh"]
path_images = data["path_images"]
camera_params = data["camera_params"]

data_chunk = create_chunk(mesh.copy(), image_name, camera_params, max_seq_len=max_seq_len, image_path=path_images, center=center, size=chunk_size, with_backtransform=True)

T_cw = camera_params[image_name]['T_cw']
_, _, T_wc = invert_pose(T_cw[:3,:3], T_cw[:3,3])

mesh_chunk_backtransformer = data_chunk["backtransformed"]
mesh_chunk = data_chunk["mesh"]
image_names_chunk = data_chunk["image_names"]
camera_params_chunk = data_chunk["camera_params"]
p_center = data_chunk["p_center"]

print(f"Chunk size: {chunk_size}")
print(f"Center: {center}")
print(f"Resolution: {resolution}")

In [ ]:
visualize_mesh(mesh_chunk_backtransformer, images=image_names_chunk, camera_params_list=camera_params_chunk.values(), opacity=1.0)

In [ ]:
data_chunk = create_chunk(mesh.copy(), image_name, camera_params, max_seq_len=max_seq_len, image_path=path_images, center=center, size=chunk_size, with_backtransform=True, with_furthest_displacement=True)

T_cw = camera_params[image_name]['T_cw']
_, _, T_wc = invert_pose(T_cw[:3,:3], T_cw[:3,3])

mesh_chunk_backtransformer = data_chunk["backtransformed"]
mesh_chunk = data_chunk["mesh"]
image_names_chunk = data_chunk["image_names"]
camera_params_chunk = data_chunk["camera_params"]
p_center = data_chunk["p_center"]

print(f"Chunk size: {chunk_size}")
print(f"Center: {center}")
print(f"Resolution: {resolution}")

In [ ]:
visualize_mesh(mesh_chunk_backtransformer, images=image_names_chunk, camera_params_list=camera_params_chunk.values(), opacity=1.0)

In [ ]:
voxel_grid, coordinates, occupancy_values = mesh_2_voxels(mesh_chunk, voxel_size=0.02, to_world_coordinates=T_wc)
print(f"Occupancy values: {occupancy_values.sum()}")
visualize_mesh(voxel_grid.as_boxes(), images=image_names_chunk, camera_params_list=camera_params_chunk.values(), opacity=1.0)

In [ ]:
from utils.chunking import mesh_2_local_voxels
from utils.transformations import invert_pose

voxel_grid_local, coordinates_local, occupancy_values_local  = mesh_2_local_voxels(mesh_chunk_backtransformer,  center, 0.02, grid_size[0])
visualize_mesh(voxel_grid_local.as_boxes(), images=image_names_chunk, camera_params_list=camera_params_chunk.values(), opacity=1.0)

In [ ]:
from utils.chunking import mesh_2_local_voxels
from utils.transformations import invert_pose


T_cw = camera_params[image_name]['T_cw']
_, _, T_wc = invert_pose(T_cw[:3,:3], T_cw[:3,3])

voxel_grid_local, coordinates_local, occupancy_values_local  = mesh_2_local_voxels(mesh_chunk,  center, 0.02, grid_size[0])
visualize_mesh(voxel_grid_local.as_boxes(), images=image_names_chunk, camera_params_list=camera_params_chunk.values(), opacity=1.0)